In [1]:
%load_ext autoreload
%autoreload 2 
import sim
import strategy as strat
import message
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from pylab import rcParams
from sortedcontainers import SortedDict
from collections import deque
import copy
rcParams['figure.figsize'] = 15, 10

In [2]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
len(data)

In [4]:
EPS = 1e-12
INF = 10**9

In [5]:
def chop_asks(mid_price, asks, money):
    chopped_asks = []
    asks.sort()
    for (price, quantity) in asks:
        if quantity > 0 + EPS and money > 0 + EPS:
            if money >= price * quantity:
                chopped_asks.append((price, quantity))
                money -= price * quantity
            else:
                chopped_asks.append((price, money / price))
                money = 0
    return chopped_asks

In [8]:
def task1(data, money):
    asks = []
    dt = copy.deepcopy(data)
    for update in dt:
        i = 0
        j = 0
        new_asks = []
        
        #Comment this for hard task
        update.order_book.bid_quantity[0] = INF 
        update.order_book.ask_quantity[0] = INF
        #
        
        while i < len(asks) and j < len(update.order_book.bid_price) and asks[i][0] < update.order_book.bid_price[j]:
            
            quantity = min(asks[i][1], update.order_book.bid_quantity[j])
            new_asks.append((update.order_book.bid_price[j], quantity))
            update.order_book.bid_quantity[j] -= quantity
            asks[i] = (asks[i][0], asks[i][1] - quantity)
            money += (update.order_book.bid_price[j] - asks[i][0]) * quantity
            
            if asks[i][1] <= 0 + EPS:
                i += 1
            if update.order_book.bid_quantity[j] <= 0 + EPS:
                j += 1
                
        asks = chop_asks(
            (update.order_book.ask_price[0] + update.order_book.bid_price[0]) / 2, 
            asks + new_asks + list(zip(update.order_book.ask_price, update.order_book.ask_quantity)), 
            money)
        
    return money

In [7]:
maximum_income(data, 100000)

245257.80612671736

In [9]:
maximum_income(data, 100000)

829620.8064028878